In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd
import json
import pymongo as pm
from glob import glob
from IPython.display import clear_output
import itertools 
from random import sample
import importlib
from PIL import Image

### set up project paths

In [ ]:
## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..'))
data_dir = os.path.abspath(os.path.join(curr_dir,'..','data')) ## use relative paths
stim_dir = os.path.abspath(os.path.join(proj_dir,'stimuli'))
analysis_dir = os.path.abspath(os.path.join(curr_dir,'..','analysis')) ## use relative paths
plot_dir =  os.path.abspath(os.path.join(curr_dir,'..','results','plots'))
csv_dir = os.path.join(proj_dir, 'results','csv')

## add helpers to python path
import sys
if os.path.join(proj_dir, 'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'utils'))
import utils as h    

animals = sorted(['rabbit','wolf','squirrel','bear','deer','owl'])
animal_paths = ['{}.png'.format(i) for i in animals]

### inspect stims and get image paths 

In [ ]:
## get stim paths
path_to_stims = os.path.abspath(os.path.join(stim_dir, 'stims'))
im_list = h.list_files(path_to_stims)
assert len(im_list)==8*len(animals)*100

In [ ]:
## sort stims in place
importlib.reload(h)
h.sort_nicely(im_list)

In [ ]:
## manually inspect the images
imsize = 256
fname = im_list[500]
print(fname)
Image.open(fname).resize((imsize,imsize))

### upload stims to s3

In [ ]:
import boto3
import botocore

In [ ]:
bucket_name = 'iternum'
full_stim_paths = h.list_files(path_to_stims)
h.sort_nicely(full_stim_paths)

In [ ]:
## helper to speed things up by not uploading images if they already exist, can be overriden 
def check_exists(s3, bucket_name, stim_name):
    try:
        s3.Object(bucket_name,stim_name).load()    
        return True
    except botocore.exceptions.ClientError as e:    
        if (e.response['Error']['Code'] == "404"):
            print('The object does not exist.')
            return False
        else:
            print('Something else has gone wrong with {}'.format(stim_name))

In [ ]:
reallyRun = 0
if reallyRun: 
    ## tell user some useful information
    print('Path to stimuli is : {}'.format(path_to_stims))
    print('Uploading to this bucket: {}'.format(bucket_name))

    ## establish connection to s3 
    s3 = boto3.resource('s3')

    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=bucket_name) 
        print('Created new bucket.')
    except:
        b = s3.Bucket(bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = False
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_stim_paths[4265:]):
        stim_name = path_to_file.split('/')[-1]
        if ((check_exists(s3, bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} | {} of {}'.format(path_to_file.split('/')[-1],(i+1),len(full_stim_paths)))
            s3.Object(bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(path_to_file.split('/')[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)

    print('Done uploading images!')